In [18]:
import numpy as np
import pandas as pd
def show_matrix(matrix):
    for i in range(len(matrix)):
        print()
        for j in range(len(matrix[i])):
            print(matrix[i][j], end='\t\t\t')

In [19]:
def epsilon_insensitive_loss(y_true, y_pred, epsilon):
    # Calculate absolute errors
    errors = np.linalg.norm(np.abs(y_true - y_pred))
    # Apply epsilon-insensitive penalty
    loss = np.maximum(0, errors - epsilon)
    return loss

def eclidean_distance(point1, point2, epsilon):
    return np.sqrt(np.sum(epsilon_insensitive_loss(point1, point2, epsilon)))

def distance(cluster1,cluster2, epsilon):
    return eclidean_distance(cluster1, cluster2, epsilon)

def min_distance(arr1, arr2, X):
    min_dist = float('inf')
    
    for idx1 in arr1:
        for idx2 in arr2:
            dist = distance(X[idx1], X[idx2], epsilon=0.1)
            if dist < min_dist:
                min_dist = dist
    
    return min_dist

def distance_mat(clusters, data):
    cluster_len = len(clusters)
    dist_mat = np.zeros((cluster_len, cluster_len))
    for i1 in range(cluster_len):
        for i2 in range(cluster_len):
            if i2 > i1:
                dist_mat[i1, i2] = min_distance(clusters[i1], clusters[i2], data)
            else : 
                dist_mat[i1, i2] = np.inf
    return dist_mat

def min_matrix(matrix):
    min_index = np.argmin(matrix)
    _, cols = matrix.shape
    i, j= min_index // cols, min_index % cols
    return i, j, matrix[i, j]
    
def linkage(clusters, data, num_clusters):
    linkage_matrix = []
    for i in range(num_clusters):
        linkage_matrix.append([])
        print(clusters)
        dist_mat = distance_mat(clusters, data)
        i_min, j_min, min_data = min_matrix(dist_mat)
        linkage_matrix[-1].append(clusters[i_min])
        linkage_matrix[-1].append(clusters[j_min])
        linkage_matrix[-1].append(min_data)
        clusters[i_min] = clusters[i_min] + clusters[j_min]
        clusters.pop(j_min)
        print(dist_mat)
    print()
    print(pd.DataFrame(linkage_matrix))
    return linkage_matrix


def compare_arr(arr1, arr2):
    for i in arr1:
        for j in arr2:
            if i == j:
                return True
    return False



def inconsistency(linkage_matrix):
    inconsistency_matrix = np.zeros(shape=(len(linkage_matrix), 4)).astype('float64')
    for i in range(len(linkage_matrix)):
        data = []
        for j in range(i,-1,-1):
            flags = np.array([
                compare_arr(linkage_matrix[i][0], linkage_matrix[j][0]),
                compare_arr(linkage_matrix[i][0], linkage_matrix[j][1]),
                compare_arr(linkage_matrix[i][1], linkage_matrix[j][0]),
                compare_arr(linkage_matrix[i][1], linkage_matrix[j][1])
                ])
            if flags.any() == True:
                    data.append(linkage_matrix[j][2])
        
        inconsistency_matrix[i][0] = np.mean(data)
        inconsistency_matrix[i][1] = np.std(data)
        inconsistency_matrix[i][2] = len(data)
        inconsistency_matrix[i][3] = 0 if inconsistency_matrix[i,1] == 0 else (linkage_matrix[i][2] - inconsistency_matrix[i,0]) / inconsistency_matrix[i,1]
    print()
    print(pd.DataFrame(inconsistency_matrix))
    # return inconsistency_matrix

In [20]:
from sklearn.datasets import make_blobs

num_clusters = 1
num_samples = 3
X, y = make_blobs(n_samples=num_samples, centers=num_clusters, cluster_std=2, random_state=42)
X += 10

In [21]:
# X = [
#     [4, 4, 1, 1],
#     [3, 0, 1, 1],
#     [8, 2, 1, 1],
#     [8, 0, 1, 1],
#     [8.5, 0, .5, .5]
# ]
np.random.seed(42)
X=np.random.normal(loc=0, scale=3, size=(10,2))
clusters = [
    [i] for i in range(10)
]
# clusters = [
#     [0], [1], [2], [3], [4]
# ]
lm = linkage(clusters, np.asarray(X), 9)
print()
inconsistency(lm)

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
[[       inf 2.21459261 1.45306107 2.03328476 1.85631248 1.71560993
  2.29774054 2.58557623 2.15121624 2.36417113]
 [       inf        inf 2.40792336 1.87038137 2.08786709 2.59505073
  3.2063278  3.06207467 2.46204836 3.14107219]
 [       inf        inf        inf 2.4727999  1.52796874 0.93682338
  2.26627824 2.11645025 1.66048119 1.99293914]
 [       inf        inf        inf        inf 2.46636897 2.65669417
  2.98114522 3.25345783 2.7917895  3.13394575]
 [       inf        inf        inf        inf        inf 1.71024091
  2.75144317 2.21745676 1.29155947 2.43112209]
 [       inf        inf        inf        inf        inf        inf
  2.17504121 1.92238662 1.66197549 1.74280535]
 [       inf        inf        inf        inf        inf        inf
         inf 2.65678463 2.75134636 1.91392591]
 [       inf        inf        inf        inf        inf        inf
         inf        inf 1.81327958 1.8538468 ]
 [       inf        inf      